In [1]:
import micropip

%pip install lib3mf

micropip.add_mock_package("py-lib3mf", "2.4.1", modules={"py_lib3mf": '''from lib3mf import *'''})

%pip install build123d --force-reinstall
%pip install yacv-server

In [2]:
from build123d import *
from yacv_server import *
from IPython.display import HTML#, JSON, Latex, Markdown
import base64

In [3]:
r = """
<html><body>
</div>
<script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
<model-viewer src="https://build123d.readthedocs.io/en/latest/_static/tea_cup.glb" alt="A tea cup modelled in build123d" camera-controls style="width: 100%; height: 60vh;"></model-viewer>
</body></html>"""
display(HTML(r))

In [4]:
# export_gltf(Box(10, 10, 10), 'box.glb', binary=True)

# with open('box.glb', 'rb') as f:
#     glb_bytes = f.read()

# base64_data = base64.b64encode(glb_bytes).decode('utf-8')

# data_uri = f'data:model/gltf-binary;base64,{base64_data}'

In [5]:
def view(obj, c1 = (0.5,0.5,0.5,1), c2 = (0,0,0.5,1), c3 = (0.5,0,0.5,1)):
    temp = b''
    for i in tessellate.tessellate(obj, c1, c2, c3).save_to_bytes():
        temp += i
    display(HTML(f"""
        <html><body>
        </div>
        <script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
        <model-viewer src="data:model/gltf-binary;base64,{base64.b64encode(temp).decode('utf-8')}" alt="A tea cup modelled in build123d" camera-controls style="width: 100%; height: 60vh;"></model-viewer>
        </body></html>"""))

In [6]:
# tess_data = tessellate.tessellate(Box(10, 10, 10).wrapped, (0.5,0.5,0.5,1), (0,0,0.5,1), (0.5,0,0.5,1)).save_to_bytes()
# tess_bin = b''
# for i in tess_data:
#     tess_bin += i

# base64_tess = base64.b64encode(tess_bin).decode('utf-8')

# data_uri = f'data:model/gltf-binary;base64,{base64_tess}'

In [7]:
# r = f"""
# <html><body>
# </div>
# <script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.min.js"></script>
# <model-viewer src="{data_uri}" alt="A tea cup modelled in build123d" camera-controls style="width: 100%; height: 60vh;"></model-viewer>
# </body></html>"""
# display(HTML(r))

In [8]:
len, wid, thickness = 70, 50, 5
inner_r, outer_r = 4, 8
gap = 0.3
hook = 10

with BuildPart() as top:
    Cylinder(outer_r, wid, rotation=(0,90,0))
    Box(wid, len, outer_r, align=(Align.CENTER, Align.MIN, Align.MAX))
    Cylinder(inner_r+gap, wid, rotation=(0,90,0), mode=Mode.SUBTRACT)
    extrude(faces().filter_by(Axis.Z)[-1], amount=-(outer_r-thickness), mode=Mode.SUBTRACT)
    fillet(faces().filter_by(Axis.Z)[-1].edges().sort_by(Axis.Y)[0], radius=0.99)
    with GridLocations(wid/5*2, 0, 3, 1):
        Box(wid/5, 2*outer_r+4, 2*outer_r, mode=Mode.SUBTRACT)
    RigidJoint("hinge", joint_location=Location(Plane.YZ.rotated((90,0,0))))

    fillet(top.edges().filter_by(Axis.Z).group_by(Axis.Y)[-1], radius=wid/2.01)
    with Locations((0,len-wid/2,0)):
        Cylinder(wid/2-10, 2*outer_r, mode=Mode.SUBTRACT)
        Box(wid/2, wid/2-10, 2*outer_r, align=(Align.MAX,Align.MAX, Align.CENTER), mode=Mode.SUBTRACT)
    fillet(top.edges(Select.LAST).filter_by(Axis.Z), radius=4)


with BuildPart() as bottom:
    Cylinder(outer_r, wid, rotation=(0,90,0))
    Box(wid, len*.75, outer_r, align=(Align.CENTER, Align.MAX, Align.MAX))
    extrude(faces().filter_by(Axis.Z)[0], amount=-(outer_r-thickness), mode=Mode.SUBTRACT)
    fillet(faces().filter_by(Axis.Z)[-1].edges().sort_by(Axis.Y)[-1], radius=0.99)
    with GridLocations(wid/5*2, 0, 2, 1):
        Box(wid/5+2*gap, 2*outer_r+4, 2*outer_r, mode=Mode.SUBTRACT)
    Cylinder(inner_r, wid, rotation=(0,90,0))
    RevoluteJoint("hinge", axis=Axis.X)

    with BuildSketch(Location((0,-len*.75,-(outer_r-thickness)))):
        RectangleRounded(hook, hook, radius=4, align=(Align.CENTER, Align.MIN))
    extrude(amount=30, dir=(0,1,1))
    fillet(bottom.edges(Select.LAST).filter_by(Plane.XY)-bottom.edges().group_by(Axis.Y)[0], 2)
    fillet(bottom.edges().group_by(Axis.Y)[0].filter_by(Axis.Z), radius=(wid-hook)/2)


bottom.part.joints["hinge"].connect_to(top.part.joints["hinge"], angle=0)

# show(top, bottom, reset_camera=Camera.KEEP, render_joints=True)
view(Compound([top.part, bottom.part]).wrapped)